In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    fletcher=False,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.8597864508628845
epoch:  1, loss: 0.8477258086204529
epoch:  2, loss: 0.8356391787528992
epoch:  3, loss: 0.8235473036766052
epoch:  4, loss: 0.8113228678703308
epoch:  5, loss: 0.7990875840187073
epoch:  6, loss: 0.7870096564292908
epoch:  7, loss: 0.774722695350647
epoch:  8, loss: 0.7623293995857239
epoch:  9, loss: 0.7502098083496094
epoch:  10, loss: 0.7378368973731995
epoch:  11, loss: 0.7255964875221252
epoch:  12, loss: 0.7133665680885315
epoch:  13, loss: 0.7011454701423645
epoch:  14, loss: 0.6889944672584534
epoch:  15, loss: 0.6768544912338257
epoch:  16, loss: 0.6647244095802307
epoch:  17, loss: 0.6526210904121399
epoch:  18, loss: 0.6405007839202881
epoch:  19, loss: 0.6283899545669556
epoch:  20, loss: 0.6163042783737183
epoch:  21, loss: 0.6042287945747375
epoch:  22, loss: 0.5921487808227539
epoch:  23, loss: 0.5800405740737915
epoch:  24, loss: 0.5678957104682922
epoch:  25, loss: 0.5557845830917358
epoch:  26, loss: 0.5436632633209229
epoch:  27, 

In [6]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.5249851094276587
Test metrics:  R2 = 0.46036979706887293


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    fletcher=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
    solver="pinv"
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.41629379987716675
epoch:  1, loss: 0.4052070677280426
epoch:  2, loss: 0.39482197165489197
epoch:  3, loss: 0.3839706480503082
epoch:  4, loss: 0.37446022033691406
epoch:  5, loss: 0.36243927478790283
epoch:  6, loss: 0.34906670451164246
epoch:  7, loss: 0.3396247923374176
epoch:  8, loss: 0.32781982421875
epoch:  9, loss: 0.3156156837940216
epoch:  10, loss: 0.30398938059806824
epoch:  11, loss: 0.29261329770088196
epoch:  12, loss: 0.281185507774353
epoch:  13, loss: 0.2699193060398102
epoch:  14, loss: 0.2587921917438507
epoch:  15, loss: 0.2477739453315735
epoch:  16, loss: 0.23698997497558594
epoch:  17, loss: 0.22624558210372925
epoch:  18, loss: 0.21583344042301178
epoch:  19, loss: 0.20580720901489258
epoch:  20, loss: 0.19489896297454834
epoch:  21, loss: 0.18509525060653687
epoch:  22, loss: 0.17524056136608124
epoch:  23, loss: 0.16535481810569763
epoch:  24, loss: 0.15594395995140076
epoch:  25, loss: 0.14653056859970093
epoch:  26, loss: 0.13774581253528

In [8]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9658597944912553
Test metrics:  R2 = 0.8913674994798716
